# 2. Working with CSV files

We're going to build on what we did in the last notebook, and focus in on playing with csv files.  There are a lot of good resources available, and I'm going to point you to a short YouTube video I liked (warning, it's a bit quirky) -- but this notebook will give you the basics.

The video is here: (https://youtu.be/Xi52tx6phRU)

## Our CSV file

Let's use the roster file that we used in the first notebook.  You'll remember we could load it using the following code:

```python
filename = "roster.csv"

with open(filename) as file_obj:   # with just a plain open command, the file will be opened for "read" access
    lines = file_obj.readlines()
```
   
If we wanted to use this method, we would have to split the lines where the commas are.  There is a better way, but let's see how we would do in raw python first

In [2]:
filename = "roster.csv"

with open(filename) as file_obj:   # with just a plain open command, the file will be opened for "read" access
    lines = file_obj.readlines()

for line in lines:
    values = line.strip().split(',')
    print(values)
    

['Name', 'Team', 'Safety Trained', 'Years', 'Grade']
['Bob', 'Programming', 'Y', '2', '11']
['Sally', 'Build', 'N', '3', '12']
['Karim', 'Safety', 'Y', '1', '9']
['Luther', 'Programming', 'N', '1', '10']
['Crystal', 'Programming', 'Y', '2', '10']
['Lex', 'Build', 'Y', '3', '12']
['Rachel', 'Safety', 'Y', '1', '11']


As you can see, *because I know all of you read the Python Crash Course*, we take the string that we read for each line, and convert it to a list that we print out.  This is the key line:

```python
values = line.strip().split(',')
```

In this line, "line" is the variable with a string (e.g. "Bob,Programming,Y,2,11\n").  The "strip()" call removes the '\n", and the "split(',')" returns a list of values ( ['Bob','Programming','Y','2','11'] ).  If that doesn't make sense, you may actually want to read the Python Crash Course!

Assuming that did make sense, it might occur to you that what we just did isn't all that helpful, as I only have the last member in the "values" line (since we were overwriting it in the loop).  Let's do something more helpful...

In [3]:
filename = "roster.csv"

with open(filename) as file_obj:   # with just a plain open command, the file will be opened for "read" access
    lines = file_obj.readlines()
    
members = []     # I'm creating an empty list for members to put the values in
    
for line in lines:
    values = line.strip().split(',')
    members.append(values)   # this will add the latest entry into the members list
    
print(members)  # the output of this is a "list of lists"

[['Name', 'Team', 'Safety Trained', 'Years', 'Grade'], ['Bob', 'Programming', 'Y', '2', '11'], ['Sally', 'Build', 'N', '3', '12'], ['Karim', 'Safety', 'Y', '1', '9'], ['Luther', 'Programming', 'N', '1', '10'], ['Crystal', 'Programming', 'Y', '2', '10'], ['Lex', 'Build', 'Y', '3', '12'], ['Rachel', 'Safety', 'Y', '1', '11']]


So you may wonder why this is useful, so let's do something with the information.  We'll do a couple of things with it, just to prove you can -- and to show a bit more syntax, but we're just scratching the surface.

In [19]:
teams = {}  # we're going to use a Python dictionary to capture the members of the sub-teams
safety = 0  # let's see how many members are trained in safety

for member in members[1:]:   # I'm using "slicing" here (e.g. [1:]) -- I want to skip the first line, which is headers
    # we are going to build a python dictionary of teams -- which means we'll create an entry for the team when we
    # first see it, and add to it after that
    
    if member[1] in teams.keys():
        teams[member[1]].append(member[0])    # we are appending a member to an existing team
    else:
        teams[member[1]]=[member[0]]          # we are adding a new team to our teams list, and creating a list with a member
        
    if member[2]=='Y':   # safety trained?
        safety = safety + 1
        
print(teams)  # let's print it and see  

# let's do a somewhat nicer print
print("\n\nFRC Sub-teams\n")

for team,team_members in teams.items():
    print(team)
    for member in team_members:
        print(f"\t{member}")

print()
        
# let's see how we're doing on Safety
print(f"We have {safety} members trained, out of {len(members)} total members")

percent_trained = (safety/len(members)*100)
print(f"{percent_trained}% are trained")

{'Programming': ['Bob', 'Luther', 'Crystal'], 'Build': ['Sally', 'Lex'], 'Safety': ['Karim', 'Rachel']}


FRC Sub-teams

Programming
	Bob
	Luther
	Crystal
Build
	Sally
	Lex
Safety
	Karim
	Rachel

We have 5 members trained, out of 8 total members
62.5% are trained


So, that gives you kind of an idea of how you can play with data using data structures and looping.  There are lots of ways to do this (and when you deal in large amounts of data there are special libraries), but hopefully you get the idea.

## Writing Data into CSV (using basic Python)

A major reason to learn how to work with files is so that all of your data is not lost when your script stops running, and one method is to write data to files.  If you write data to a CSV file, aside from being able to use what you've learned to read it, you can also open it in a spreadsheet -- which can be a major advantage.

If you are writing a lot of data, and want to analyze it later, it can be very helpful to be able to look at it in a spreadsheet.

There are many ways to accomplish this in basic Python.  Below I'll demonstrate one way to create a csv file, but keep in mind there are many ways (we'll look at another using the csv module).

In the sample below, I'm going to generate some "fake data" (I'll use a random number generator to simulate readings from a sensor).  This is a really trivial example, but the mechanics of saving the data would work for anything.

In [30]:
import random     # we'll use a function from here to generate our sensor data
import datetime   # we'll timestamp our sensor data

# let's create (define) a function to simulate a sensor reading

def read_sensor():
    read_time = "{}".format(datetime.datetime.now())  # this is another way to format a string
    
    read_data = random.normalvariate(100,5)
    return(read_time,read_data)

# First, let's generate a small dataset (100 items), and then write it to a file

small_data_set = []   # create an empty list to append to
small_data_set.append(("Datetime","Sensor_Reading"))   # adding a header row

for i in range(100):
    small_data_set.append(read_sensor())  # read_sensor returns a 2 value tuple (datetime, reading), which we'll add to this list
    
# now let's write this to a file
with open('small_data_set.csv','w') as sds_out:
    for line in small_data_set:
        formatted_line = ",".join(map(str,line))  # what the heck?  
                                                  #   Okay...  We want to join the values in the line into a single string
                                                  #            with commas inbetween.  So we would write ",".join(line) 
                                                  #            this would create a string with all the values in line, with a
                                                  #            comma between them.  But our sensor data is a float, not a string
                                                  #            so we are going to convert everything to a string, which is 
                                                  #            what "map(str,line)" does.  It will "map" each value in line to
                                                  #            the function "str" -- which makes everything a string...
        sds_out.write(f"{formatted_line}\n")  # write it to a file, and add a "newline" 


Okay, there is a lot going on in the code above, and I'll explain a bit -- but if you have questions, reach out to me on Slack (@johnrad), or find me at a meeting.  

```python
def read_sensor():
```
This is a function definition.  Once we define a function, we can call it from other parts of the code.  In this case, we will call it 100 times in a loop to generate some sensor data to write to a file

```python
for i in range(100):
    small_data_set.append(read_sensor())
```
This is a loop.  The range function generates numbers, in this case it will go from 0 to 99.  This is a very common use of range to create a loop that will run 100 times.  

small_data_set is a Python list, and we can add a new element to the list with "append".  You can call the "read_sensor" function (as we are) inside of append.  What happens is that "read_sensor" is called, and returns a two element "tuple" (like a list, but cannot be changed), which is then appended to the small_data_set list.

If this is a bit confusing, just stick with it, believe it or not the ability to do this stuff is why people love Python so much!  In the end, we have a python list (small_data_set) that contains 101 2 value tuples (a header, and 100 sensor readings).

```python
for line in small_data_set:
        formatted_line = ",".join(map(str,line))
        ```
This is the last code snippet I'll explain here.  Hopefully you're getting used to the "for xxxx in xxxx:" syntax.  Here, small_data_set has 101 elements, and we want to write each out to a file.  The for loop will keep processing them one at a time until its done.

The next line is a bit tricky, but it helps to remember what we are trying to do.  In each case we have a tuple ('2017-12-10', 10.25) and we want a string we can write to a file "2017-12-10, 10.25".  Python strings have a "join" method that will join things using the string, so if we write ",".join(("jack","jill","pail")), we will get "jack,jill,pail".

This is great, but what is the "map" thing?  Well, the join function can only work on strings, and our sensor data is a number (a "float", to be precise).  We can convert it with the "str" function.  map(str, line) will "map" the str function to everything in "line"  (I'm sure this makes perfect sense!).

Let me give a quick example.  Suppose you have a list of numbers num_list, like this [ 2.0, 3, 5.15, 6.8 ].

If I call "str_list = map(str, num_list)", it will return [ '2.0', '3', '5.15', '6.8' ], basically returning a list of strings.  Note that the other list (num_list) is still there, and still numbers -- this call creates a new one.

The proof is on the file system though -- go ahead and find the "small_data_set.csv" file, and open it in either an editor or a spreadsheet (or both).  

This is kind of a pain though, and you'd think it would be easier all around -- and it is!  We're going to look next at the csv module and how it helps make all of this a lot easier.